# Homework and bake-off: Relation extraction using distant supervision

In [1]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [2]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [3]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [4]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [5]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [6]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [7]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [8]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [9]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [10]:
featurizers = [simple_bag_of_words_featurizer]

In [11]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [12]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.841      0.388      0.682        340       5716
author                    0.784      0.534      0.717        509       5885
capital                   0.607      0.179      0.411         95       5471
contains                  0.800      0.597      0.749       3904       9280
film_performance          0.790      0.569      0.733        766       6142
founders                  0.801      0.392      0.663        380       5756
genre                     0.595      0.147      0.370        170       5546
has_sibling               0.842      0.246      0.568        499       5875
has_spouse                0.865      0.313      0.640        594       5970
is_a                      0.669      0.227      0.482        497       5873
nationality               0.625      0.166      0.403        301       5677
parents     

Studying model weights might yield insights:

In [13]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.544 Córdoba
     2.430 Taluks
     2.262 Valais
     ..... .....
    -1.145 who
    -1.550 Europe
    -1.664 America

Highest and lowest feature weights for relation author:

     2.934 author
     2.613 wrote
     2.374 books
     ..... .....
    -2.620 Alice
    -6.069 dystopian
    -6.974 1865

Highest and lowest feature weights for relation capital:

     2.720 capital
     1.737 km
     1.658 capitals
     ..... .....
    -1.219 and
    -1.241 North
    -1.566 ’

Highest and lowest feature weights for relation contains:

     2.889 third-largest
     2.431 bordered
     2.279 County
     ..... .....
    -2.264 Malaysian
    -2.355 band
    -2.402 who

Highest and lowest feature weights for relation film_performance:

     4.170 starring
     3.729 co-starring
     3.475 alongside
     ..... .....
    -1.965 Malice
    -1.966 Wonderland
    -2.165 Iruvar

Highest and lowest feature weights for relation founders:

    

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [14]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [15]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [16]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))                
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [17]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],    
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.833      0.471      0.722        340       5716
author                    0.832      0.446      0.709        509       5885
capital                   0.519      0.147      0.345         95       5471
contains                  0.650      0.410      0.582       3904       9280
film_performance          0.804      0.321      0.618        766       6142
founders                  0.722      0.239      0.515        380       5756
genre                     0.500      0.059      0.200        170       5546
has_sibling               0.821      0.238      0.551        499       5875
has_spouse                0.855      0.357      0.668        594       5970
is_a                      0.720      0.135      0.386        497       5873
nationality               0.648      0.189      0.436        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [18]:
from sklearn.svm import SVC
def run_svm_model_factory():
    
    ##### YOUR CODE HERE
    
#     return rel_ext.experiment(
#         splits,
#         train_split='train',
#         test_split='dev',
#         featurizers=[glove_middle_featurizer],    
#         model_factory=lambda: SVC(kernel='linear', max_iter=3000),
#         vectorize=False,
#         verbose=True)
    return rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=[simple_bag_of_words_featurizer],    
        model_factory=lambda: SVC(kernel='linear', max_iter=3000),
        verbose=True)



In [19]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."    

In [35]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/_

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.298      0.318      0.302        340       5716
author                    0.259      0.835      0.301        509       5885
capital                   0.605      0.274      0.487         95       5471
contains                  0.428      0.916      0.479       3904       9280
film_performance          0.199      0.808      0.235        766       6142
founders                  0.459      0.500      0.467        380       5756
genre                     0.597      0.253      0.469        170       5546
has_sibling               0.117      0.804      0.141        499       5875
has_spouse                0.238      0.818      0.278        594       5970
is_a                      0.152      0.672      0.179        497       5873
nationality               0.300      0.262      0.292        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [20]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter): 
    # Append these to the end of the keys you add/access in 
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass 
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word+subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word+object_subject_suffix] += 1
    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
directional_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[directional_bag_of_words_featurizer],
    verbose=True)

print('Number of model features:', directional_results['models']['adjoins'].coef_.shape[1])

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.826      0.418      0.691        340       5716
author                    0.809      0.591      0.754        509       5885
capital                   0.677      0.221      0.479         95       5471
contains                  0.810      0.674      0.779       3904       9280
film_performance          0.857      0.640      0.802        766       6142
founders                  0.767      0.408      0.652        380       5756
genre                     0.741      0.235      0.518        170       5546
has_sibling               0.816      0.240      0.552        499       5875
has_spouse                0.886      0.352      0.679        594       5970
is_a                      0.765      0.256      0.547        497       5873
nationality               0.653      0.219      0.468        301       5677
parents     

In [21]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [22]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [23]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for bg in get_tag_bigrams(ex.middle_POS):
            feature_counter[bg] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for bg in get_tag_bigrams(ex.middle_POS):
            feature_counter[bg] += 1
    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each 
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"
    
    ##### YOUR CODE HERE
    tags = [start_symbol]+get_tags(s)+[end_symbol]
    bigrams = []
    for i in range(len(tags)-1):
        bigrams.append(tags[i]+' '+tags[i+1])
    return bigrams

    
def get_tags(s): 
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.    
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)  

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
directional_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[middle_bigram_pos_tag_featurizer],
    verbose=True)



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.815      0.350      0.644        340       5716
author                    0.711      0.348      0.588        509       5885
capital                   0.556      0.158      0.369         95       5471
contains                  0.766      0.587      0.722       3904       9280
film_performance          0.697      0.444      0.625        766       6142
founders                  0.621      0.168      0.404        380       5756
genre                     0.660      0.182      0.433        170       5546
has_sibling               0.636      0.164      0.404        499       5875
has_spouse                0.740      0.273      0.551        594       5970
is_a                      0.581      0.173      0.395        497       5873
nationality               0.489      0.076      0.235        301       5677
parents     

In [24]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [25]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [26]:
# import nltk
# nltk.download('wordnet')
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for bg in get_synsets(ex.middle_POS):
            feature_counter[bg] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for bg in get_synsets(ex.middle_POS):
            feature_counter[bg] += 1
    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets 
    associated with elements of `s`.
    """   
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]
    
    ##### YOUR CODE HERE
    return [str(syn) for word, pos in wt for syn in wn.synsets(word, pos=convert_tag(pos))]
    
    
def convert_tag(t):
    """Converts tags so that they can be used by WordNet:
    
    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """        
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None    


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
directional_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[synset_featurizer],
    verbose=True)



/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.793      0.326      0.617        340       5716
author                    0.789      0.442      0.682        509       5885
capital                   0.607      0.179      0.411         95       5471
contains                  0.776      0.588      0.729       3904       9280
film_performance          0.772      0.557      0.717        766       6142
founders                  0.764      0.384      0.638        380       5756
genre                     0.478      0.194      0.370        170       5546
has_sibling               0.784      0.218      0.517        499       5875
has_spouse                0.856      0.300      0.624        594       5970
is_a                      0.676      0.243      0.499        497       5873
nationality               0.446      0.136      0.306        301       5677
parents     

In [27]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():   
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [28]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [29]:
# Enter your system description in this cell.
"""
For our final solution we used a feature ensemble of bidirectional bag of words and trigram level POS bag of words. 
This net us an F-score of .65 (P=0.798, R=0.397) using Logistic Regression. 
We tried other feature combinations of n-gram level POS strings, glove embeddings, and average or bag of lengths middle length featurization.
We tried an SVM classifier with an rbf kernel, however it only reached an F-score of .53 (due to poor recall) even with 15000 max iterations.
"""
def middle_trigram_pos_tag_featurizer(kbt, corpus, feature_counter):
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for bg in get_tag_trigrams(ex.middle_POS):
            feature_counter[bg] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for bg in get_tag_trigrams(ex.middle_POS):
            feature_counter[bg] += 1
    return feature_counter


def get_tag_trigrams(s):
    """Suggested helper method for `middle_trigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each 
    element is a POS trigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"
    
    ##### YOUR CODE HERE
    tags = [start_symbol]+get_tags(s)+[end_symbol]
    trigrams = []
    for i in range(len(tags)-2):
        trigrams.append(tags[i]+' '+tags[i+1]+' '+tags[i+2])
    return trigrams
directional_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[middle_trigram_pos_tag_featurizer, directional_bag_of_words_featurizer],
    verbose=True)

if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass

# My peak score was: 0.65\n

# Please do not remove this comment.

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.841      0.421      0.701        340       5716
author                    0.840      0.631      0.788        509       5885
capital                   0.710      0.232      0.502         95       5471
contains                  0.825      0.678      0.791       3904       9280
film_performance          0.869      0.695      0.828        766       6142
founders                  0.822      0.437      0.699        380       5756
genre                     0.818      0.371      0.659        170       5546
has_sibling               0.836      0.255      0.574        499       5875
has_spouse                0.903      0.359      0.692        594       5970
is_a                      0.774      0.330      0.610        497       5873
nationality               0.637      0.216      0.458        301       5677
parents     

'My peak score was: .65\n'

In [39]:
# SVM attempt
# directional_results = rel_ext.experiment(
#     splits,
#     train_split='train',
#     test_split='dev',
#     featurizers=[middle_trigram_pos_tag_featurizer, directional_bag_of_words_featurizer],
#     model_factory=lambda: SVC(max_iter=15000),
#     verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.862      0.350      0.667        340       5716
author                    0.880      0.546      0.784        509       5885
capital                   0.462      0.063      0.204         95       5471
contains                  0.853      0.606      0.789       3904       9280
film_performance          0.903      0.560      0.805        766       6142
founders                  0.903      0.268      0.613        380       5756
genre                     0.829      0.171      0.468        170       5546
has_sibling               0.919      0.182      0.508        499       5875
has_spouse                0.892      0.264      0.605        594       5970
is_a                      0.864      0.205      0.526        497       5873
nationality               0.763      0.150      0.419        301       5677
parents     

In [54]:
# Only Trigram features

# directional_results = rel_ext.experiment(
#     splits,
#     train_split='train',
#     test_split='dev',
#     featurizers=[middle_trigram_pos_tag_featurizer],
#     verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.828      0.353      0.652        340       5716
author                    0.754      0.350      0.613        509       5885
capital                   0.484      0.158      0.342         95       5471
contains                  0.740      0.608      0.709       3904       9280
film_performance          0.759      0.492      0.684        766       6142
founders                  0.672      0.216      0.472        380       5756
genre                     0.711      0.159      0.419        170       5546
has_sibling               0.714      0.160      0.422        499       5875
has_spouse                0.759      0.271      0.558        594       5970
is_a                      0.679      0.217      0.477        497       5873
nationality               0.468      0.096      0.264        301       5677
parents     

In [58]:
# 4 gram features
# def middle_4gram_pos_tag_featurizer(kbt, corpus, feature_counter):
    
#     ##### YOUR CODE HERE
#     for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
#         for bg in get_tag_4grams(ex.middle_POS):
#             feature_counter[bg] += 1
#     for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
#         for bg in get_tag_4grams(ex.middle_POS):
#             feature_counter[bg] += 1
#     return feature_counter


# def get_tag_4grams(s):
#     """Suggested helper method for `middle_trigram_pos_tag_featurizer`.
#     This should be defined so that it returns a list of str, where each 
#     element is a POS trigram."""
#     # The values of `start_symbol` and `end_symbol` are defined
#     # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
#     start_symbol = "<s>"
#     end_symbol = "</s>"
    
#     ##### YOUR CODE HERE
#     tags = [start_symbol]+get_tags(s)+[end_symbol]
#     ngrams = []
#     for i in range(len(tags)-3):
#         ngrams.append(tags[i]+' '+tags[i+1]+' '+tags[i+2]+' '+tags[i+3])
#     return ngrams

# directional_results = rel_ext.experiment(
#     splits,
#     train_split='train',
#     test_split='dev',
#     featurizers=[middle_4gram_pos_tag_featurizer],
#     verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.783      0.382      0.647        340       5716
author                    0.809      0.582      0.750        509       5885
capital                   0.541      0.211      0.412         95       5471
contains                  0.786      0.627      0.748       3904       9280
film_performance          0.790      0.638      0.754        766       6142
founders                  0.769      0.421      0.660        380       5756
genre                     0.697      0.312      0.559        170       5546
has_sibling               0.820      0.275      0.587        499       5875
has_spouse                0.892      0.362      0.690        594       5970
is_a                      0.685      0.320      0.558        497       5873
nationality               0.548      0.189      0.397        301       5677
parents     

In [63]:
# # bigram and directional_bag_of_words
# directional_results = rel_ext.experiment(
#     splits,
#     train_split='train',
#     test_split='dev',
#     featurizers=[middle_bigram_pos_tag_featurizer, directional_bag_of_words_featurizer],
#     verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.867      0.424      0.717        340       5716
author                    0.848      0.637      0.795        509       5885
capital                   0.697      0.242      0.507         95       5471
contains                  0.808      0.703      0.785       3904       9280
film_performance          0.844      0.684      0.806        766       6142
founders                  0.860      0.437      0.720        380       5756
genre                     0.764      0.324      0.600        170       5546
has_sibling               0.906      0.251      0.595        499       5875
has_spouse                0.885      0.364      0.688        594       5970
is_a                      0.800      0.330      0.623        497       5873
nationality               0.701      0.226      0.493        301       5677
parents     

In [67]:
# bag of middle length featurizer
# def middle_length_featurizer(kbt, corpus, feature_counter):
    
#     ##### YOUR CODE HERE
#     total = 0
#     count = 0
#     for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
#         feature_counter[str(len(ex.middle.split()))] += 1
# #         total += len(ex.middle.split())
# #         count += 1
#     for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
#         feature_counter[str(len(ex.middle.split()))] += 1
# #         total += len(ex.middle.split())
# #         count += 1
# #     if count == 0:
# #         div = 0
# #     else:
# #         div = total/count
# #     feature_counter['average_length'] += div
#     return feature_counter
# directional_results = rel_ext.experiment(
#     splits,
#     train_split='train',
#     test_split='dev',
#     featurizers=[middle_trigram_pos_tag_featurizer, directional_bag_of_words_featurizer, middle_length_featurizer],
#     verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.857      0.424      0.711        340       5716
author                    0.846      0.639      0.795        509       5885
capital                   0.636      0.221      0.463         95       5471
contains                  0.805      0.703      0.782       3904       9280
film_performance          0.847      0.688      0.810        766       6142
founders                  0.863      0.432      0.719        380       5756
genre                     0.764      0.324      0.600        170       5546
has_sibling               0.906      0.253      0.597        499       5875
has_spouse                0.883      0.357      0.682        594       5970
is_a                      0.817      0.332      0.632        497       5873
nationality               0.629      0.219      0.458        301       5677
parents     

In [70]:
#Unvectorized GLoVE featurizer combined with middle trigram and directional bag of words
# def glove_slow_featurizer(kbt, corpus, feature_counter, np_func=np.sum):
#     reps = []
#     for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
#         for word in ex.middle.split():
#             rep = glove_lookup.get(word)
#             if rep is not None:
#                 reps.append(rep)
#     # A random representation of the right dimensionality if the
#     # example happens not to overlap with GloVe's vocabulary:
#     if len(reps) == 0:
#         dim = len(next(iter(glove_lookup.values())))                
#         vec = utils.randvec(n=dim)
#     else:
#         vec = np_func(reps, axis=0)
    
#     feature_counter.update({'glove_feat_'+str(i): v for i, v in enumerate(vec.flatten().tolist())})
#     return feature_counter
# directional_results = rel_ext.experiment(
#     splits,
#     train_split='train',
#     test_split='dev',
#     featurizers=[glove_slow_featurizer, middle_trigram_pos_tag_featurizer, directional_bag_of_words_featurizer],
#     verbose=True)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.845      0.432      0.709        340       5716
author                    0.828      0.625      0.778        509       5885
capital                   0.619      0.274      0.494         95       5471
contains                  0.795      0.754      0.786       3904       9280
film_performance          0.841      0.698      0.808        766       6142
founders                  0.797      0.445      0.688        380       5756
genre                     0.743      0.324      0.590        170       5546
has_sibling               0.862      0.275      0.604        499       5875
has_spouse                0.854      0.375      0.681        594       5970
is_a                      0.715      0.328      0.578        497       5873
nationality               0.563      0.252      0.452        301       5677
parents     

In [29]:
#GLoVE featurizers trying to capture directionality
# def glove_middle_forward_reverse_featurizer(kbt, corpus, np_func=np.sum):
#     reps = []
#     for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
#         for word in ex.middle.split():
#             rep = glove_lookup.get(word)
#             if rep is not None:
#                 reps.append(rep)
#     reps2 = []
#     for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
#         for word in ex.middle.split():
#             rep = glove_lookup.get(word)
#             if rep is not None:
#                 reps2.append(rep)
#     # A random representation of the right dimensionality if the
#     # example happens not to overlap with GloVe's vocabulary:
#     if len(reps) == 0:
#         dim = len(next(iter(glove_lookup.values())))                
#         v1 = utils.randvec(n=dim)
#     else:
#         v1 = np_func(reps, axis=0)
        
#     if len(reps2) == 0:
#         dim = len(next(iter(glove_lookup.values())))                
#         v2 = utils.randvec(n=dim)
#     else:
#         v2 = np_func(reps2, axis=0)
        
#     return np.concatenate((v1,v2), axis=-1)
    
# def glove_middle_both_featurizer(kbt, corpus, np_func=np.sum):
#     reps = []
#     for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
#         for word in ex.middle.split():
#             rep = glove_lookup.get(word)
#             if rep is not None:
#                 reps.append(rep)
#     for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
#         for word in ex.middle.split():
#             rep = glove_lookup.get(word)
#             if rep is not None:
#                 reps.append(rep)
#     # A random representation of the right dimensionality if the
#     # example happens not to overlap with GloVe's vocabulary:
#     if len(reps) == 0:
#         dim = len(next(iter(glove_lookup.values())))                
#         return utils.randvec(n=dim)
#     else:
#         return np_func(reps, axis=0)
    
# glove_results = rel_ext.experiment(
#     splits,
#     train_split='train',
#     test_split='dev',
#     featurizers=[glove_middle_reverse_featurizer],    
#     vectorize=False, # Crucial for this featurizer!
#     verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.773      0.462      0.681        340       5716
author                    0.807      0.583      0.750        509       5885
capital                   0.548      0.358      0.496         95       5471
contains                  0.800      0.662      0.768       3904       9280
film_performance          0.812      0.614      0.762        766       6142
founders                  0.697      0.437      0.623        380       5756
genre                     0.440      0.235      0.375        170       5546
has_sibling               0.760      0.255      0.544        499       5875
has_spouse                0.767      0.387      0.641        594       5970
is_a                      0.634      0.219      0.460        497       5873
nationality               0.512      0.216      0.402        301       5677
parents     

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [31]:
# Enter your bake-off assessment code in this cell. 
rel_ext_data_home_test = os.path.join(
    rel_ext_data_home, 'bakeoff-rel_ext-test-data')
bakeoff_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[middle_trigram_pos_tag_featurizer, directional_bag_of_words_featurizer],
    verbose=True)
rel_ext.bake_off_experiment(bakeoff_results, rel_ext_data_home_test)
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.841      0.421      0.701        340       5716
author                    0.840      0.631      0.788        509       5885
capital                   0.710      0.232      0.502         95       5471
contains                  0.825      0.678      0.791       3904       9280
film_performance          0.869      0.695      0.828        766       6142
founders                  0.822      0.437      0.699        380       5756
genre                     0.818      0.371      0.659        170       5546
has_sibling               0.836      0.255      0.574        499       5875
has_spouse                0.903      0.359      0.692        594       5970
is_a                      0.774      0.330      0.610        497       5873
nationality               0.637      0.216      0.458        301       5677
parents     

In [32]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported 
# by the code above. Please enter only a number between 
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
    0.636

